# Experiment Name: King’s Super Math Saga Experiment

**Author:** Jing Jiang

**E-mail:** jingjiang0224@gmail.com

## 1 Problem Statement

**Overview:** *Super Math Saga* is a fictional mobile game, where players beat levels by answering a math question correctly to progress through a map. The game is Free to Play and players can optionally buy hints on the levels.

**Experiment:** In this A/B Test experiment, two groups of users are offered two different game experiences.

**Objective:** The purpose of conducting this experiment is to determine which version makes Super Math Saga a better game.
The objectives is to increase user engagement within the game and boost revenue at the same time.

## 2 Metric Choice

**Success Metrics**

*   Average Purchases Per Player
*   Average Game-ends Per Player


**Guardrail Metrics**

*   Daily Active User
(Due to the significant difference in sample sizes between the control and experimental groups, this metric is not suitable for comparison in this scenario.)
*   Retention Rate
*   First-Time Purchase Conversion Rate






## 3 Experiment Hypothesis

Since I have chosen two primary metrics for this case, I have formulated two hypotheses.

**Experiment Hypothesis 1:**

Implementing this new change in the gaming experience for a select group of users will increase "Average Game-ends Per Player".

**Experiment Hypothesis 2:**

Implementing this new change in the gaming experience for a select group of users will increase "Average Purchases Per Player".



## 4 Design of Experiment

**4.1 Create Variations:**


*   **Group A (Control Group):** Remain original setup.
*   **Group B (Experiment Group):** Expose to new experience.

**4.2 Determine Sample Size:** The sample size has already been determined for this experiment. So I conducted a simple re-check to confirm the size.

**Conclution:**

*   The number of players in Group A: 8265610 (80%)
*   The number of players in Group B: 2065446 (20%)

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.power import TTestIndPower
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
# Parameter Configuration

# Set up account credentials
auth.authenticate_user()
print('Authenticated')

# Set up project info
client = bigquery.Client(project='king-ds-recruit-candidate-1011')

assignment="king-ds-recruit-candidate-1011.abtest.assignment"
activity="king-ds-recruit-candidate-1011.abtest.activity"
experiment_user_activity="king-ds-recruit-candidate-1011.abtest.experiment_user_activity"

# Set use_legacy_sql=false
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = False

# function to execute query SQL
def execute_query(query):
    # print(f"Executing query: {query}")
    res=client.query(query, job_config)
    return res.result()


Authenticated


In [ ]:
# Check the proportion of data sizes in both groups

res=execute_query(f"""
    SELECT
        abtest_group,
        COUNT(DISTINCT playerid) AS num
    FROM {assignment}
    GROUP BY
        abtest_group
""")

sample_size = res.to_dataframe()
print("The number of users in each group:")
display(sample_size)

The number of users in each group:


,abtest_group,num
0,B,2065446
1,A,8265610


**4.3 Randomly Assign Users:**

As mentioned, players were randomly assigned into two groups. So I conducted a simple re-check to confirm the randomization.


**Conclusion:**

The distribution of new and old players in both groups is consistent.

In [ ]:
# Check whether the distribution of new and old users is consistent between the experimental group and the control group.

## Metrics Definition
# New Player: Those whose "install_date" >= "assignment_date"
# Old Player: Those whose "install_date" < "assignment_date"

res = execute_query(f"""
    SELECT
        abtest_group,
        new_player + old_player AS total_player,
        new_player,
        old_player,
        new_player / (new_player + old_player) AS new_player_rate,
        old_player / (new_player + old_player) AS old_player_rate
    FROM
        (SELECT
            abtest_group,
            MAX(CASE WHEN player_type = 'new' THEN player_num ELSE NULL END) AS new_player,
            MAX(CASE WHEN player_type = 'old' THEN player_num ELSE NULL END) AS old_player
        FROM
            (SELECT
                abtest_group,
                player_type,
                COUNT(DISTINCT playerid) AS player_num
              FROM
                  (SELECT
                      playerid,
                      abtest_group,
                      if(install_date < assignment_date, "old", "new") as player_type
                  FROM {assignment}
                  ) tmp1
              GROUP BY
                  abtest_group,
                  player_type
              ) tmp2
          GROUP BY
              abtest_group
      )tmp3
""")

user_type_distribution = res.to_dataframe()
print("The user distribution in each group:")
display(user_type_distribution)


The user distribution in each group:


,abtest_group,total_player,new_player,old_player,new_player_rate,old_player_rate
0,A,8265610,1489573,6776037,0.180213,0.819787
1,B,2065446,373361,1692085,0.180765,0.819235


**4.4 Experiment Duration**

**Conclusion:**


*   The experiment lasted from 2017-05-04 to 2017-05-22, 19 days in total.
*   The experiment started on Thursday and ending on Monday three weeks later.

In [ ]:
# Experiment Duration Checking

res = execute_query(f"""
    SELECT
        assignment_date,
        FORMAT_DATE('%A', PARSE_DATE('%Y-%m-%d', assignment_date)) AS day_of_week
    FROM {assignment}
    GROUP BY
        assignment_date,
        FORMAT_DATE('%A', PARSE_DATE('%Y-%m-%d', assignment_date))
    ORDER BY
        assignment_date ASC
""")

duration = res.to_dataframe()
print("Experiment Duration:")
display(duration)


Experiment Duration:


,assignment_date,day_of_week
0,2017-05-04,Thursday
1,2017-05-05,Friday
2,2017-05-06,Saturday
3,2017-05-07,Sunday
4,2017-05-08,Monday
5,2017-05-09,Tuesday
6,2017-05-10,Wednesday
7,2017-05-11,Thursday
8,2017-05-12,Friday
9,2017-05-13,Saturday


## 5 Online Validation

**5.1 Check Mixed Assignment:** Check if there was any players that were assigned to both groups


In [ ]:
# Data Cleaning

# Check Mixed Assignment: Check if there was any players that were assigned to both groups
res = execute_query(f"""
    SELECT
        playerid,
        COUNT(DISTINCT abtest_group) AS assign_group_cnt
    FROM {assignment}
    GROUP BY
        playerid
    HAVING
        COUNT(DISTINCT abtest_group) > 1
""")

mix_assignment = res.to_dataframe()
print("Mix Assignment:")
display(mix_assignment)

Mix Assignment:


,playerid,assign_group_cnt


**5.2 Check Multiple Assignment:** Check if there was any players that were assigned for multiple times.

In [ ]:
# Check Multiple Assignment: Check if there was any players that were assigned for multiple times.
res = execute_query(f"""
    SELECT
        playerid,
        COUNT(DISTINCT assignment_date) AS assign_date_cnt
    FROM {assignment}
    GROUP BY
        playerid
    HAVING
        COUNT(DISTINCT abtest_group) > 1
""")

multi_assignment = res.to_dataframe()
print("Multiple Assignment:")
display(multi_assignment)


Multiple Assignment:


,playerid,assign_date_cnt


## 6 Run Experiment

## 7 Collect Data and Analyze Results

In [ ]:
# Create a new table named 'experiment_user_activity' to include basic user information along with their corresponding activity data during the experiment period.

# Note: Ensure that each player's activity is recorded only after they have been assigned to the corresponding group.

execute_query(f"""
    CREATE TABLE king-ds-recruit-candidate-1011.abtest.experiment_user_activity AS
    SELECT
        assignment_table.playerid AS playerid,
        abtest_group,
        IF(install_date < assignment_date, "old", "new") AS player_type,
        assignment_date,
        install_date,
        conversion_date,
        activity_date,
        purchases,
        gameends
    FROM
        (SELECT
            playerid,
            abtest_group,
            assignment_date,
            install_date,
            conversion_date
        FROM {assignment}
        ) AS assignment_table
    LEFT JOIN
        (SELECT
            playerid,
            activity_date,
            purchases,
            gameends
        FROM {activity}
        ) AS activity_table
    ON
        assignment_table.playerid = activity_table.playerid
    WHERE
        activity_date >= assignment_date       ## Ensure that each player's activity is recorded only after they have been assigned to the corresponding group.
    AND
        activity_date <= '2017-05-22'
""")

### **Metrics Calculation**

In [ ]:
# Metrics Calculation

# avg_gameends: total gameends / the number of players who played at least one gameend
# purchase_conversion: the number of players who made at least one purchases during experiment / the number of players who played at least one gameend
# avg_purchase: total purchases / the number of players who played at least one gameend
# avg_purchase: total purchases / the number of players who have made at least one purchase
# first_purchase_conversion: the number of players who made the first purchase during experiment / the number of players who played at least one gameend


activity_info = execute_query(f"""
    SELECT
        abtest_group,
        AVG(avg_gameends) AS avg_gameends,
        AVG(purchase_conversion) AS purchase_conversion,
        AVG(avg_purchase_per_player) AS avg_purchase_per_player,
        AVG(avg_purchase_per_paid_player) AS avg_purchase_per_paid_player,
        AVG(first_purchase_conversion) AS first_purchase_conversion
    FROM
        (# Calculate metric per activity date
        SELECT
            activity_date,
            abtest_group,
        COUNT(DISTINCT IF(gameends != 0, playerid, null)) AS play_cnt, # number of players who played at least one gameend
        COUNT(DISTINCT IF(purchases != 0, playerid, null)) AS purchase_cnt,  #number of paid users
        SUM(purchases) AS sum_purchases,   # total purchases
        SUM(gameends) AS sum_gameends,   # total gameends

        SUM(gameends) / COUNT(DISTINCT IF(gameends != 0, playerid, null)) AS avg_gameends, # average gameends per player

        COUNT(DISTINCT IF(purchases != 0, playerid, null)) / COUNT(DISTINCT IF(gameends != 0, playerid, null)) AS purchase_conversion, # purchase conversion rate
        SUM(purchases) / COUNT(DISTINCT IF(gameends != 0, playerid, null)) AS avg_purchase_per_player, # average purchase per player
        SUM(purchases) / COUNT(DISTINCT IF(purchases != 0, playerid, null)) AS avg_purchase_per_paid_player, # average purchase per paid player
        COUNT(DISTINCT IF(conversion_date >= assignment_date, playerid, null)) / COUNT(DISTINCT IF(gameends != 0, playerid, null)) AS first_purchase_conversion, # first purchase conversion rate

        FROM king-ds-recruit-candidate-1011.abtest.experiment_activity_info
        GROUP BY
            activity_date,
            abtest_group
        ) tmp
    GROUP BY
        abtest_group
""")



In [ ]:
activity_info = activity_info.to_dataframe()
print("Metric Calculation:")
display(activity_info)

Metric Calculation:


,abtest_group,avg_gameends,purchase_conversion,avg_purchase_per_player,avg_purchase_per_paid_player,first_purchase_conversion
0,A,13.18507,0.012102,0.030451,2.516168,0.002220
1,B,12.93509,0.012446,0.032661,2.623387,0.002247


### **Hypothesis Testing**

In [ ]:
# Description: Average number of gameends and purchases per player per active day

# avg_gameends_per_day: total gameends / the number of days they played game
# avg_purchases_per_day: total purchases / the number of days they played game


player_info = execute_query(f"""
    SELECT
        playerid,
        abtest_group,
        COUNT(DISTINCT IF(gameends!=0, activity_date, NULL)) AS played_days, # the number of days player played
        IF(COUNT(DISTINCT IF(gameends!=0, activity_date, NULL)) = 0, 0, SUM(gameends) / COUNT(DISTINCT IF(gameends!=0, activity_date, NULL))) AS avg_gameends_per_day,
        IF(COUNT(DISTINCT IF(gameends!=0, activity_date, NULL)) = 0, 0, SUM(purchases) / COUNT(DISTINCT IF(gameends!=0, activity_date, NULL))) AS avg_purchases_per_day

    FROM
        king-ds-recruit-candidate-1011.abtest.experiment_activity_info
    GROUP BY
        playerid,
        abtest_group

""")



**Hypothesis Testing for avg_purchases_per_user**

In [ ]:
player_info = player_info.to_dataframe()

Group_A_avg_purchases = player_info[player_info['abtest_group'] == 'A']['avg_purchases_per_day'].to_numpy()
Group_B_avg_purchases = player_info[player_info['abtest_group'] == 'B']['avg_purchases_per_day'].to_numpy()

In [ ]:
# Perform Levene test for equal variances.
stats.levene(Group_A_avg_purchases, Group_B_avg_purchases)

LeveneResult(statistic=11.032377880685903, pvalue=0.0008953453779476625)

In [ ]:
# Calculate the means and standard deviations
mean_control_purchases = np.mean(Group_A_avg_purchases)
mean_treatment_purchases = np.mean(Group_B_avg_purchases)
std_control_purchases = np.std(Group_A_avg_purchases, ddof=1)
std_treatment_purchases = np.std(Group_B_avg_purchases, ddof=1)

print(f"Mean of control group: {mean_control_purchases}")
print(f"Mean of treatment group: {mean_treatment_purchases}")
print(f"Standard deviation of control group: {std_control_purchases}")
print(f"Standard deviation of treatment group: {std_treatment_purchases}")

# Perform the two-sample t-test
t_stat_purchases, p_value_purchases = ttest_ind(Group_A_avg_purchases, Group_B_avg_purchases, alternative='less', equal_var=False)

print(f"T-statistic: {t_stat_purchases}")
print(f"P-value: {p_value_purchases}")

Mean of control group: 0.030649776273500236
Mean of treatment group: 0.032561412063420184
Standard deviation of control group: 0.7363962585939153
Standard deviation of treatment group: 0.7535039971841592
T-statistic: -3.276041293751688
P-value: 0.0005263715693935271


**Calculate statistical power**

In [ ]:
## calculate power

# Calculate pooled standard deviation
pooled_std = np.sqrt((std_control_purchases**2 + std_treatment_purchases**2) / 2)

# Calculate effect size (Cohen's d)
effect_size = (mean_treatment_purchases - mean_control_purchases) / pooled_std

# Define parameters
alpha = 0.05
n_control = len(Group_A_avg_purchases)
n_test = len(Group_B_avg_purchases)
# nobs2 = nobs1 * ratio
ratio = n_test / n_control

# Calculate power
power_analysis = TTestIndPower()
power = power_analysis.solve_power(effect_size=effect_size, nobs1=n_control, alpha=alpha, ratio=ratio, alternative='larger')

print(f"Effect size (Cohen's d): {effect_size}")
print(f"Power of the test: {power}")

Effect size (Cohen's d): 0.002565956711638915
Power of the test: 0.9509040220694777


**Hypothesis Testing for avg_gameends_per_user**

In [ ]:
Group_A_avg_gameends = player_info[player_info['abtest_group'] == 'A']['avg_gameends_per_day'].to_numpy()
Group_B_avg_gameends = player_info[player_info['abtest_group'] == 'B']['avg_gameends_per_day'].to_numpy()

In [ ]:
# Perform Levene test for equal variances.
stats.levene(Group_A_avg_gameends, Group_B_avg_gameends)

LeveneResult(statistic=180.1862892717096, pvalue=4.4166288993838516e-41)

In [ ]:
# Calculate the means and standard deviations
mean_control_gameends = np.mean(Group_A_avg_gameends)
mean_treatment_gameends = np.mean(Group_B_avg_gameends)
std_control_gameends = np.std(Group_A_avg_gameends, ddof=1)
std_treatment_gameends = np.std(Group_B_avg_gameends, ddof=1)

print(f"Mean of control group: {mean_control_gameends}")
print(f"Mean of treatment group: {mean_treatment_gameends}")
print(f"Standard deviation of control group: {std_control_gameends}")
print(f"Standard deviation of treatment group: {std_treatment_gameends}")

# Perform the two-sample t-test
t_stat_gameends, p_value_gameends = ttest_ind(Group_A_avg_gameends, Group_B_avg_gameends, alternative='less')

print(f"T-statistic: {t_stat_gameends}")
print(f"P-value: {p_value_gameends}")

Mean of control group: 13.183939908560097
Mean of treatment group: 12.950402384517725
Standard deviation of control group: 9.687666318858252
Standard deviation of treatment group: 9.505092717202533
T-statistic: 31.105451894251082
P-value: 1.0


In [ ]:
# Perform the two-sample t-test
t_stat_gameends, p_value_gameends = ttest_ind(Group_A_avg_gameends, Group_B_avg_gameends, alternative='two-sided')

print(f"T-statistic: {t_stat_gameends}")
print(f"P-value: {p_value_gameends>0.05}")

T-statistic: 31.105451894251082
P-value: False


**Calculate statistical power**

In [ ]:
## calculate power
n_control = len(Group_A_avg_gameends)
n_test = len(Group_B_avg_gameends)

# Calculate pooled standard deviation
#pooled_std = np.sqrt((std_control_gameends**2 + std_treatment_gameends**2) / 2)
pooled_std = np.sqrt(((n_control - 1) * (std_control_gameends**2) + (n_test - 1) * (std_treatment_gameends**2)) / (n_control + n_test - 2))

# Calculate effect size (Cohen's d)
effect_size = abs((mean_treatment_gameends - mean_control_gameends)) / pooled_std

# Define parameters
alpha = 0.05
n_control = len(Group_A_avg_gameends)
n_test = len(Group_B_avg_gameends)
# nobs2 = nobs1 * ratio
ratio = n_test / n_control

# Calculate power
power_analysis = TTestIndPower()
power = power_analysis.solve_power(effect_size=effect_size, nobs1=n_control, alpha=alpha, ratio=ratio, alternative='larger')

print(f"Effect size (Cohen's d): {effect_size}")
print(f"Power of the test: {power==1}")
print(n_control, ratio, mean_treatment_gameends, mean_control_gameends)

Effect size (Cohen's d): 0.024197165528133863
Power of the test: True
8265610 0.24988427956315384 12.950402384517725 13.183939908560097


**Hypothesis Testing for purchased_conversion_rate**

In [ ]:
# Description: Get number of players who have purchased and first purchased during experiment.

# purchased_player: players whose purchases > 0
# first_purchased_player: players whose purchases > 0 and first paid during experiment (conversion_date>=assignment_date)


player_purchase_info = execute_query(f"""
    SELECT
        abtest_group,
        SUM(played_user) AS played_user,
        SUM(purchased_player) AS purchased_player,
        SUM(first_purchased_player) AS first_purchased_player
    FROM
        (SELECT
            abtest_group,
            activity_date,
            COUNT(DISTINCT IF(gameends!=0, playerid, NULL)) AS played_user, # the number of user who have played at lease one round
            COUNT(DISTINCT IF(purchases!=0, playerid, null)) AS purchased_player,
            COUNT(DISTINCT IF(((purchases!=0) AND (conversion_date>=assignment_date)), playerid, null)) AS first_purchased_player
        FROM
            king-ds-recruit-candidate-1011.abtest.experiment_activity_info
        GROUP BY
            abtest_group,
            activity_date
            ) t
    GROUP BY
        abtest_group

""")



In [ ]:
player_purchase_info = player_purchase_info.to_dataframe()

display(player_purchase_info)

,abtest_group,played_user,purchased_player,first_purchased_player
0,A,99402081,1202997,95661
1,B,24828576,308992,24908


In [ ]:
# Calculate Purchase Rate
player_purchase_info['purchased_rate'] = player_purchase_info['purchased_player'] / player_purchase_info['played_user']

# Calculate First Purchase Rate
player_purchase_info['first_purchased_rate'] = player_purchase_info['first_purchased_player'] / player_purchase_info['played_user']

In [ ]:
player_purchase_info

,abtest_group,played_user,purchased_player,first_purchased_player,purchased_rate,first_purchased_rate
0,A,99402081,1202997,95661,0.012102,0.000962
1,B,24828576,308992,24908,0.012445,0.001003


In [ ]:
# Sample conversion counts and sizes for two groups
count1 = np.array(player_purchase_info['purchased_player'])
nobs = np.array(player_purchase_info['played_user'])

# Perform the proportions z-test
stat1, pval1 = proportions_ztest(count1, nobs)

print(f"Z-statistic-1: {stat1}")
print(f"P-value-1: {pval1}")

Z-statistic-1: -13.929988038940198
P-value-1: 4.1642522964113317e-44


In [ ]:
# Sample conversion counts and sizes for two groups
count2 = np.array(player_purchase_info['first_purchased_player'])
nobs = np.array(player_purchase_info['played_user'])

# Perform the proportions z-test
stat2, pval2 = proportions_ztest(count2, nobs)

print(f"Z-statistic-2: {stat2}")
print(f"P-value-2: {pval2}")

Z-statistic-2: -5.84516024758707
P-value-2: 5.060808452165329e-09


**Hypothesis Testing for avg_purchases_per_new_user**

In [ ]:
# Description: Calculate average purchases for each new player.

# new player: install_date>=assignment_date


new_player_info = execute_query(f"""
    SELECT
        abtest_group,
        playerid,
        COUNT(DISTINCT IF(gameends!=0, activity_date, NULL)) AS played_days,

        IF(COUNT(DISTINCT IF(gameends!=0, activity_date, NULL)) = 0, 0, SUM(purchases) / COUNT(DISTINCT IF(gameends!=0, activity_date, NULL))) AS avg_purchase_new_player

    FROM
        king-ds-recruit-candidate-1011.abtest.experiment_activity_info
    WHERE install_date>=assignment_date
    GROUP BY
        abtest_group,
        playerid

""")



In [ ]:
new_player_info = new_player_info.to_dataframe()

Group_A_avg_purchases_new = new_player_info[new_player_info['abtest_group'] == 'A']['avg_purchase_new_player'].to_numpy()
Group_B_avg_purchases_new = new_player_info[new_player_info['abtest_group'] == 'B']['avg_purchase_new_player'].to_numpy()

In [ ]:
# Perform Levene test for equal variances.
stats.levene(Group_A_avg_purchases_new, Group_B_avg_purchases_new)

LeveneResult(statistic=0.03995925107114626, pvalue=0.8415602968727129)

In [ ]:
# Calculate the means and standard deviations
mean_control_purchases_new = np.mean(Group_A_avg_purchases_new)
mean_treatment_purchases_new = np.mean(Group_B_avg_purchases_new)
std_control_purchases_new = np.std(Group_A_avg_purchases_new, ddof=1)
std_treatment_purchases_new = np.std(Group_B_avg_purchases_new, ddof=1)

print(f"Mean of control group: {mean_control_purchases_new}")
print(f"Mean of treatment group: {mean_treatment_purchases_new}")
print(f"Standard deviation of control group: {std_control_purchases_new}")
print(f"Standard deviation of treatment group: {std_treatment_purchases_new}")

# Perform the two-sample t-test
t_stat_purchases, p_value_purchases = ttest_ind(Group_A_avg_purchases_new, Group_B_avg_purchases_new, alternative='two-sided')

print(f"T-statistic: {t_stat_purchases}")
print(f"P-value: {p_value_purchases}")

Mean of control group: 0.030973004669318967
Mean of treatment group: 0.03124572304658684
Standard deviation of control group: 0.7408775799026045
Standard deviation of treatment group: 0.7632847217816547
T-statistic: -0.1998981017197168
P-value: 0.8415602969085942


## 8 Draw Conclusion

Thus, Group B makes Super Math Saga a better game.
Because it shows statistically higher Average Purchases Per User compared to Group A, indicating a more effective monetization strategy despite similar levels of game engagement.